In [13]:
normalizer = Normalizer()
clustered_dong = pd.read_json("./data/군집상권데이터.json")
menu_data = pd.read_json("./data/메뉴_추상설명_재료_토큰화.json")
cafe_menu_data = pd.read_json("./data/메뉴 데이터.json")

In [7]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
import Levenshtein
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
import Levenshtein
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def connect_mysql():
    try:
        mydb = mysql.connector.connect(
            host="project-db-campus.smhrd.com",
            user="campus_24K_LI_p2_3",
            password="smhrd3",
            database="campus_24K_LI_p2_3",
            port=3307
        )
        return mydb
    except Error as e:
        print("데이터베이스 연결 실패")
        return None

# 연결 종료
def close_connection(connection):
    if connection.is_connected():
        connection.close()
        print("MySQL 연결 종료.")

# db의 데이터를 데이터 프레임으로 받아옴
def fetch_data_from_table(table_name):
    try:
        connection = connect_mysql()
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name}"  # 조회할 테이블 지정
        cursor.execute(query)
        results = cursor.fetchall()  # 모든 데이터를 가져오기

        # 열 이름 가져오기
        column_names = [desc[0] for desc in cursor.description]
        db_df = pd.DataFrame(results, columns=column_names)
        close_connection(connection)
        return db_df

    except Error as e:
        print("데이터 조회 실패:", e)
        close_connection(connection)
        return None

# Levenshtein 거리 기반 유사도 계산
def levenshtein_similarity(str1, str2):
    # Levenshtein 거리 계산
    distance = Levenshtein.distance(str1, str2)
    # 유사도를 0~1 사이 값으로 변환 (1 - 정규화된 거리)
    max_len = max(len(str1), len(str2))
    return 1 - (distance / max_len)

# 입력 메뉴와 가장 유사한 메뉴 찾기
def find_similar_menu(data, input_menu, threshold=0.5):
    # 메뉴 리스트 가져오기
    menu_list = data["menu"].tolist()
    
    # 각 메뉴에 대해 Levenshtein 유사도 계산
    similarities = [levenshtein_similarity(input_menu, menu) for menu in menu_list]
    
    # 유사도가 가장 높은 메뉴 찾기
    max_similarity = max(similarities)
    print(max_similarity)
    most_similar_index = np.argmax(similarities)
    
    # 임계값 비교
    if max_similarity < threshold:
        return "None"
    else:
        return menu_list[most_similar_index]


# 유사 상권 탐색
def yusa_dong(clustered_dong, dong_pk):
    target_dong_index = clustered_dong[clustered_dong["법정동 번호(PK)"]==dong_pk].index[0]

    same_cluster_dong = clustered_dong[clustered_dong["군집"]==clustered_dong.iloc[target_dong_index,-1]]
    same_cluster_dong.reset_index(drop=True, inplace=True)
    target_dong_index = same_cluster_dong[same_cluster_dong["법정동 번호(PK)"]==dong_pk].index[0]
    
    simil_data = same_cluster_dong.iloc[:,3:-1]
    simil_data_scaled = normalizer.fit_transform(simil_data)

    train_tensor = torch.tensor(simil_data_scaled, dtype=torch.float64)
    euclidean_distances = torch.cdist(train_tensor[target_dong_index].unsqueeze(0), train_tensor).squeeze()
    top_idx = np.argsort(euclidean_distances)[1:10]

    return [same_cluster_dong["동 이름"].iloc[int(idx)] for idx in top_idx]



def data_tokenizing(data):
    data["ingred"] = data["ingred"].replace('\'', '', regex=True)
    kiwi = Kiwi()
    for i in range(len(data["info"])):
        data.iloc[i, 1] = kiwi_tokenizer(data["info"][i], tokenizer=kiwi)
    return data


def ingred_sim(data, menu_name, num=10):
    # info_simil 함수로 유사한 메뉴 추출
    info_sim = info_simil(data, menu_name, 50)
    
    # 유사한 메뉴가 없는 경우 처리
    if info_sim.empty or len(info_sim["menu"]) == 0:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # TF-IDF 변환
    menu_type = [menu.split(" ")[-1]for menu in info_sim["menu"]]
    
    tfidfv = TfidfVectorizer().fit_transform(info_sim["ingred"]+menu_type)

    # 코사인 유사도 계산
    ingred_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    # 자기 자신 제외하고 유사도 순으로 정렬
    ingred_simil_index = np.argsort(ingred_simil)[::-1][1:num + 1]

    # 유사도가 낮은 경우 처리
    if len(ingred_simil_index) == 0 or ingred_simil[ingred_simil_index[0]] < 0.2:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # 결과 반환
    return list(info_sim.iloc[ingred_simil_index]["menu"])


def info_simil(data, menu_name, num=10):
    # 메뉴 이름이 유효한지 확인
    if menu_name not in data["menu"].values:
        print("메뉴 이름이 잘못되었습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 선택된 메뉴의 인덱스 가져오기
    pick_idx = data[data["menu"] == menu_name].index[0]

    # TF-IDF 변환
    tfidfv = TfidfVectorizer().fit_transform(data["tokenized_info"])

    # 코사인 유사도 계산
    cosine_simil = cosine_similarity(tfidfv[pick_idx], tfidfv)[0]

    # 유사도를 기준으로 정렬 (내림차순)
    simil_index = np.argsort(cosine_simil)[::-1]

    # 자기 자신 제외하고 상위 num개 선택
    simil_index = simil_index[1:num + 1]

    # 유사도가 일정 수준 이하인 경우 처리
    if len(simil_index) == 0 or cosine_simil[simil_index[0]] < 0.2:
        print("유사한 메뉴가 존재하지 않습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 결과 반환
    return data.iloc[simil_index]


def input_simil(data, input_data, num=10):
    input_token = kiwi_tokenizer(input_data)
    text_vector = np.array(data["menu"] + " " + data["tokenized_info"])
    t_vector = np.insert(text_vector, 0, input_token)

    tfidfv = TfidfVectorizer().fit_transform(t_vector)
    cosine_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    simil_index = np.flip(np.argsort(cosine_simil[1:]))  # 자기 자신은 제외

    # 유사도 제한
    threshold = 0.15
    if cosine_simil[simil_index[0] + 1] < threshold:
        return ["유사한 메뉴가 존재하지 않습니다..."]
    else:
        top_matches = data.iloc[simil_index[:num], 0]
        return list(top_matches)


def kiwi_tokenizer(sentence, tokenizer=Kiwi(typos="basic")):
    sentence = sentence.replace(" ", "")
    stopwords = Stopwords()
    get_tags = ["XR", "NNG", "VA"]
    sentence = tokenizer.space(sentence)

    kiwi_result = tokenizer.tokenize(
        sentence, normalize_coda=True, stopwords=stopwords)
    token_list = []
    for token in range(len(kiwi_result)):
        if (kiwi_result[token].tag in get_tags):
            token_list.append(kiwi_result[token].form)
    token_str = " ".join(token_list)
    return token_str


# 키워드 추천 분석 결과 전달
def keyword_menu_send_to_spring(menu_data, keyword, recommanded):
    spring_url = "http://localhost:8087/api/receive-keyword-menu"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in recommanded]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in recommanded]
    except:
        menu_info = []
        menu_ingred = []
    
    payload = {
        "status" : "success",
        "keyword": keyword,
        "recommanded": recommanded,
        "menuInfo": menu_info,
        "menuIngred":menu_ingred
    }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    print(payload)
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        # 예외처리
        print(f"스프링 부트 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}


# 분석한 유사 상권 정보 전달
def similar_dong_send_to_spring(similar_dongs):
    spring_url = "http://localhost:8087/api/receive-similar-dongs"
    # 스프링에 dong이라는 이름으로 dong_name 데이터 전송
    payload = {"similarDongs": similar_dongs,
               "status": "success"
              }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()  # HTTP 오류 있는지 확인 => 예외 발생
        return payload
    except Exception as e:
        # 예외처리
        print(f"스프링 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}


# 유사 메뉴 분석 결과 스프링으로 전송
def similar_menu_recommanded(menu_data, similar_menus, menu_name):
    spring_url="http://localhost:8087/api/receive-similar-menus"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in similar_menus]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in similar_menus]
    except:
        menu_info = []
        menu_ingred=[]
    
    payload={"status":"success",
            "similarMenus":similar_menus,
            "menu": menu_name,
             "menuInfo": menu_info,
             "menuIngred":menu_ingred
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}


# 상권 공통 메뉴 분석 결과 스프링으로 전송
def search_sanggueon_gongtong(menu_data, gongtong_menus, sang1, sang2, sang1_ratio, sang2_ratio):
    spring_url="http://localhost:8087/api/receive-sanggueon-gongtong"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in gongtong_menus if menu in list(menu_data["menu"])]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in gongtong_menus if menu in list(menu_data["menu"])]
    except:
        menu_info = []
        menu_ingred = []
    
    payload={"status":"success",
            "gongtongMenus" : gongtong_menus,
             "sang1" : sang1,
             "sang2":sang2,
             "sang1MenuCnt":sang1_ratio,
             "sang2MenuCnt":sang2_ratio,
             "menuInfo":menu_info,
             "menuIngred":menu_ingred
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}


# 상권2만 언급이 많은 메뉴 전송
def search_sanggueon_only(menu_data, only_menus, sang1, sang2, sang1_ratio, sang2_ratio):
    spring_url="http://localhost:8087/api/recieve-sanggueon-only"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in only_menus if menu in list(menu_data["menu"])]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in only_menus if menu in list(menu_data["menu"])]
    except:
        menu_info = []
        menu_ingred = []
    
    payload={"status":"success",
             "onlyMenus":only_menus,
             "sang1":sang1,
             "sang2":sang2,
             "sang1MenuCnt":sang1_ratio,
             "sang2MenuCnt":sang2_ratio,
             "menuInfo":menu_info,
             "menuIngred":menu_ingred
            }
    headers={"Content-Type":"application/json"}
    print(payload)
    
    try:
        response=requests.post(spring_url,json=payload,headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status":"error", "error":str(e)}


# 좌표를 통해 법정동 번호 조회
def get_dong_number(latitude, longitude, dong_data=clustered_dong):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    response = requests.get(url, headers=headers)

    # 200번 : 통신 성공
    if response.status_code == 200:
        address_info = response.json()
        # 카카오 api에 해당 좌표 정보가 있는지 확인
        if address_info['documents']:
            addr = address_info['documents'][0]['address']['address_name']
            region = addr.split(" ")[0]
            gu = addr.split(" ")[1]
            dong = addr.split(" ")[2]

            region_dong = dong_data[dong_data["지역"].str.contains(region)]
            dong_number = list(
                region_dong[region_dong["동 이름"] == dong].head(1)["법정동 번호(PK)"])[0]
            return dong_number
    return None


# 두 상권 중 다른 상권에만 유독 더 많이 파는 메뉴
def yusasang_only(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argsort(sang1_rat)[:])
    sang2_most_rat = np.flip(np.argsort(sang2_rat)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]

    only_sang2 = []
    sang1_ratio = []
    sang2_ratio = []
    popularity = len(sang1_most)//5
    
    for i in range(len(sang2_most)):
        if(sang2_most[i] not in sang1_most[:popularity]):
            only_sang2.append(sang2_most[i])
            try:
                sang1_ratio.append(sang1_cnt_list[sang1_menu.index(sang2_most[i])])
            except:
                sang1_ratio.append(0)
            sang2_ratio.append(sang2_cnt_list[sang2_most_rat[i]])
        if(len(only_sang2) == 5):
            break
    if(len(only_sang2) < 3):
        only_sang2.append("두 상권의 거의 모든 메뉴가 유사합니다.")
        sang1_ratio.append(0)
        sang2_ratio.append(0)

    return [only_sang2, sang1_ratio, sang2_ratio]


#두 상권 모두 많이 파는 메뉴
def yusasang_gongtong(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argsort(sang1_rat)[:])
    sang2_most_rat = np.flip(np.argsort(sang2_rat)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]
    
    sang1_ratio = []
    sang2_ratio = []
    popularity = len(sang2_most) // 10
    
    gongtong_sang = []
    for i in range(popularity, len(sang1_most)):
        if(sang1_most[i] in sang2_most[:popularity*2]):
            gongtong_sang.append(sang1_most[i])
            sang1_ratio.append(sang1_cnt_list[sang1_most_rat[i]])
            sang2_ratio.append(sang2_cnt_list[sang2_menu.index(sang1_most[i])])
        if(len(gongtong_sang)==5):
            break
    if(len(gongtong_sang) < 3):
        gongtong_sang.append("두 상권의 메뉴는 거의 유사하지 않습니다.")
    print(sang1_ratio)
    print(sang2_ratio)
    return [gongtong_sang, sang1_ratio, sang2_ratio]


# 현재 상권에서 주로 파는 메뉴
def sang_menu(data, sang, no_menus=[]):
    sangdata = data[data["dong"]==sang]
    
    sang_count = Counter(sangdata['menu_keyword'])
    sang_most = sang_count.most_common(len(sang_count))
    
    sang_best_menu = []
    for i in range(5, len(sang_most)):
        if(sang_most[i][0] in no_menus):
            continue
        sang_best_menu.append(sang_most[i][0])
        if(len(sang_best_menu)==5):
            break
    if(len(sang_best_menu)<3):
        sang_best_menu.append("해당 상권은 분석할 수 없습니다. 사유 : 상권 내 카페 수 부족")
    return sang_best_menu


app = Flask(__name__)
CORS(app)  # 모든 도메인에서의 요청 허용

# CORS(app, resources={r"/api/*": {"origins": "http://localhost:8087"}})
# REST API key
KAKAO_API_KEY = "015412fbc48c3a4e31b1926b6adb667e"


# 스프링 부트로부터 사용자가 클릭한 좌표 정보를 전달받아 동을 찾은 후 분석하고 전송
@app.route('/send-coordinates', methods=['POST'])
def receive_coordinates():
    # 클라이언트로부터 받은 데이터(현 위치 위도, 경도)
    data = request.get_json()
    latitude = data.get('latitude')
    longitude = data.get('longitude')
    
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization" : f"KakaoAK {KAKAO_API_KEY}"}
    
    response = requests.get(url, headers=headers)
    
    
    dong_number = get_dong_number(latitude, longitude)
    if(dong_number == None):
        return jsonify({"status" : "error", "message" : "현재 지역의 법정동 번호를 찾을 수 없습니다."})
    
    # 유사 상권의 법정동 번호
    similar_dongs = yusa_dong(clustered_dong, dong_number)
    
    # 스트링 부트로 전달
    spring_response = similar_dong_send_to_spring(similar_dongs)
    # 전송 결과 확인
    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({
            "status" : "error",
            "yusadongs" : "",
            "message" : spring_response.get("error")
        })


# 유사 상권 탐색 2. 스프링으로부터 동을 전달받아 유사한 동 분석
@app.route("/yusa-dong-search", methods=["POST"])
def receive_dong():
    data = request.get_json()
    input_data = data.get("dong").strip()
    dong = "-1"
    region = "-1"
    print(input_data)
    
    yusa_dong_list = []
    
    if(len(input_data.split(" "))>1):
        dong = input_data.split(" ")[-1]
        region = input_data.split(" ")[0]
    else:
        dong = input_data
    
    if(dong not in list(clustered_dong["동 이름"])):
        yusa_dong_list.append("None")
        return jsonify({"status":"success",
                        "dong":"None",
                        "yusaDongNum":"None"
                        })
        
    else:
        # 혹시 지역까지 입력해주면 더 확실하게 동을 찾아 줄 수 있음(동 이름이 겹치는 경우가 많음...)
        if(clustered_dong["지역"].str.contains(region).sum()>0):
            region_data = clustered_dong[clustered_dong["지역"].str.contains(region)]
            dong_data = region_data[region_data["동 이름"]==dong]
        else:
            dong_data = clustered_dong[clustered_dong["동 이름"]==dong]
    
        dong_pk = int(dong_data["법정동 번호(PK)"].values[0])
        yusa_dong_number = yusa_dong(clustered_dong, dong_pk)
        
        payload ={"status":"success",
                 "dong":dong,
                 "yusaDongNum":yusa_dong_number}
        
        return jsonify(payload)
        
        
    

# 스프링 부트로부터 키워드 전달받아 분석
@app.route("/analyze-keyword", methods=["POST"])
def analyze_keyword():
    # 요청 본문 데이터 확인 : 키워드
    data = request.get_json()
    keyword = data.get("keyword", "")
    recommanded = input_simil(menu_data, keyword)

    spring_response = keyword_menu_send_to_spring(menu_data, keyword, recommanded)

    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

# 스프링 부트로부터 메뉴 전달받아 분석
@app.route("/analyze-menu", methods=["POST"])
def analyze_menu():
    # 요청 데이터 확인 : 메뉴
    data = request.get_json()
    menu_name = data.get("menu","")
    
    similar_menu_name="None"
    similar_menus = []
    # 사용자 추가 메뉴의 경우 메뉴 데이터에 있을 수도 있고 없을 수도 있음
    # 따라서 메뉴 데이터에 없는 경우를 대비해야 함
    if(menu_name in list(menu_data["menu"])):
        similar_menus = ingred_sim(menu_data, menu_name)
    else:
        similar_menu_name = find_similar_menu(menu_data, menu_name)
        if(similar_menu_name == "None"):
            print("None")
            # 유사한 메뉴 데이터가 없을 경우 키워드 추천 알고리즘으로 추천
            similar_menus=input_simil(menu_data, menu_name)
        else:
            # 유사한 메뉴 데이터가 있는 경우 그 메뉴에 대한 유사 메뉴 추천
            print(similar_menu_name)
            
            similar_menus = ingred_sim(menu_data, similar_menu_name)
    
    spring_response = similar_menu_recommanded(menu_data, similar_menus, menu_name)
    spring_response["menu"]=menu_name
    spring_response["changeMenu"]=similar_menu_name

    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-gongtong", methods=["POST"])
def sanggueon_gongtong():
    data = request.get_json()
    sang1 = data.get("sang1", "")
    sang2 = data.get("sang2", "")
    gongtong_menus, sang1_cnt, sang2_cnt = yusasang_gongtong(cafe_menu_data, sang1, sang2)
    print(gongtong_menus)
    spring_response = search_sanggueon_gongtong(menu_data, gongtong_menus,sang1, sang2, sang1_cnt, sang2_cnt)
    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-only", methods=["POST"])
def sanggueon_only():
    data=request.get_json()
    sang1=data.get("sang1","")
    sang2=data.get("sang2","")
    only_menus, sang1_cnt, sang2_cnt = yusasang_only(cafe_menu_data, sang1, sang2)
    print(only_menus)
    spring_response = search_sanggueon_only(menu_data, only_menus, sang1, sang2, sang1_cnt, sang2_cnt)
    print(spring_response)
    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error","message":spring_response.get("error")})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.219.245:5001
Press CTRL+C to quit
127.0.0.1 - - [06/Jan/2025 09:48:55] "POST /analyze-menu HTTP/1.1" 200 -


0.5
딸기 마카롱
{'status': 'success', 'similarMenus': ['커스터드 파이', '딸기 아이스크림', '베리 크림 도넛', '딸기 생크림 케이크', '딸기 크럼블', '딸기 스콘', '딸기 파이', '딸기 크레페', '딸기 샌드위치', '딸기 머핀'], 'menu': '마카롱', 'menuInfo': ['[부드러움, 달콤함, 바삭한 크러스트, 크리미한 커스터드 필링, 클래식 디저트, 고급스러움, 만족감, 화사함, 깔끔함, 사랑스러움]', '[새콤달콤, 부드러움, 시원함, 신선한 딸기, 크리미한 질감, 고급 디저트, 여름 감성, 만족감, 생동감, 화려함]', '[새콤달콤, 부드러운 크림, 촉촉한 도넛, 신선한 베리 풍미, 화려한 디저트, 고급스러움, 만족감, 사랑스러움, 화사함, 생동감]', '[달콤함, 상큼함, 사랑스러움, 신선한 딸기, 부드러운 생크림, 가벼운 식감, 새콤달콤, 생동감 있는 맛, 화사함, 만족스러움]', '[새콤달콤, 바삭한 크럼블 토핑, 촉촉한 딸기 필링, 화려한 디저트, 고급스러움, 만족감, 사랑스러움, 화사함, 창의적, 생동감]', '[새콤달콤, 부드러움, 상큼함, 촉촉한 식감, 신선한 딸기, 클래식 디저트, 만족감, 화사함, 고급스러움, 사랑스러움]', '[새콤달콤, 바삭한 크러스트, 부드러운 딸기 속, 신선한 풍미, 클래식 디저트, 고급스러움, 만족감, 화사함, 생동감, 사랑스러움]', '[새콤달콤, 부드러움, 얇은 크레페, 신선한 딸기 필링, 크리미한 크림, 화려한 디저트, 만족감, 사랑스러움, 화사함, 생동감]', '[새콤달콤, 부드러움, 신선한 딸기, 크리미한 크림, 촉촉한 빵, 건강한 디저트, 만족감, 사랑스러움, 화사함, 자연주의]', '[새콤달콤, 부드러운 질감, 촉촉한 속, 신선한 딸기 풍미, 클래식 디저트, 고급스러움, 만족감, 사랑스러움, 화사함, 생동감]'], 'menuIngred': ["[ '파이 반죽', '계란', '설탕', '우유', '바닐라 익스트랙', '소금', '버터', '전분

127.0.0.1 - - [06/Jan/2025 09:48:59] "POST /analyze-menu HTTP/1.1" 200 -


0.8
애플 파이
{'status': 'success', 'similarMenus': ['복숭아 타르트', '펌킨 타르트', '얼그레이 타르트', '애플 시나몬 타르트', '딸기 타르트', '밤 타르트', '오렌지 타르트', '레몬 타르트', '애플 시나몬 데니쉬', '샤인머스켓 타르트'], 'menu': '애플파이', 'menuInfo': ['[달콤함, 상큼함, 바삭한 크러스트, 부드러운 복숭아 속, 신선한 풍미, 여름 감성, 고급스러움, 만족감, 화려함, 사랑스러움]', '[담백함, 달콤함, 바삭한 타르트지, 부드러운 호박 필링, 건강한 선택, 가을 감성, 고급스러운, 만족감, 촉촉함, 자연주의]', '[향긋함, 달콤함, 바삭한 크러스트, 부드러운 얼그레이 크림, 고급 디저트, 클래식 감성, 만족감, 깔끔함, 화사함, 독특함]', '[상큼함, 향긋함, 바삭한 크러스트, 부드러운 사과와 시나몬 필링, 가을 감성, 클래식 디저트, 만족감, 따뜻함, 풍미 깊음, 화사함]', '[새콤달콤, 바삭한 크러스트, 부드러운 크림, 신선한 딸기 풍미, 클래식 디저트, 만족감, 화사함, 사랑스러움, 고급스러움, 생동감]', '[고소함, 달콤함, 바삭한 크러스트, 촉촉한 밤 필링, 전통적 감성, 건강한 디저트, 만족감, 따뜻함, 자연주의, 고급스러움]', '[상큼함, 달콤함, 바삭한 크러스트, 신선한 오렌지 필링, 건강한 디저트, 고급스러움, 만족감, 화사함, 자연주의, 사랑스러움]', '[상큼함, 달콤함, 바삭한 크러스트, 신선한 레몬 필링, 건강한 디저트, 자연주의, 만족감, 깔끔함, 화사함, 사랑스러움]', '[바삭함, 상큼함, 향긋한 시나몬, 부드러운 사과 필링, 건강한 디저트, 가을 감성, 고급스러움, 만족감, 화사함, 따뜻함]', '[상큼함, 달콤함, 바삭한 크러스트, 신선한 샤인머스켓, 고급 디저트, 여름 감성, 만족감, 화사함, 사랑스러움, 생동감]'], 'menuIngred': ["[ '타르트지', '복숭아', '버터', '설탕', '계란', '밀가루', '바

...

127.0.0.1 - - [06/Jan/2025 10:33:11] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:33:12] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:34:14] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:34:14] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:35:20] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:35:21] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:36:44] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:36:44] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:38:27] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:38:27] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:38:52] "POST /analyze-keyword HTTP/1.1" 200 -


{'status': 'success', 'keyword': '상큼', 'recommanded': ['애플 주스', '청포도 아이스티', '블루베리 샌드위치', '라임 타르트', '요거 프라페', '블루베리 크림 케이크', '레몬 티', '라임 파이', '과일 타르트', '청귤 크림 케이크'], 'menuInfo': ['[상큼함, 과일향, 건강함, 자연스러움, 활기참, 달콤한 과일향, 자연스러운 상큼함, 신선함, 부드러운 끝맛]', '[상큼함, 과일향, 청량감, 여름, 개운함, 상큼한 과일향, 자연스러운 단맛, 새콤달콤, 여름의 생기]', '[상큼함, 과일의 풍미, 달콤함, 신선한 블루베리, 촉촉한 식감, 새콤달콤, 고급 디저트, 건강 간식, 이국적, 가벼운]', '[상큼함, 산뜻함, 크리미함, 바삭한 타르트지, 라임의 풍미, 고급 디저트, 클래식함, 만족감, 화사함, 여름 감성]', '[상큼함, 건강함, 부드러움, 새콤달콤, 크리미함, 크리미한 질감, 새콤달콤한 감미, 건강한 맛, 상쾌함, 자연스러운 조화]', '[상큼함, 달콤함, 부드러움, 신선한 블루베리, 촉촉한 케이크, 고급 디저트, 만족감, 화사함, 생동감, 사랑스러움]', '[상큼함, 따뜻함, 깔끔함, 건강함, 리프레시, 상큼한 시트러스, 은은한 달콤함, 건강감, 청량한 여운]', '[상큼함, 산뜻함, 크리미함, 바삭한 파이 크러스트, 라임의 독특한 풍미, 여름 감성, 고급스러운, 만족감, 화사함, 풍부함]', '[상큼함, 달콤함, 바삭한 크러스트, 신선한 과일, 크리미한 필링, 클래식 디저트, 고급스러움, 만족감, 화려함, 생동감]', '[상큼함, 산뜻함, 부드러움, 촉촉한 식감, 청귤의 풍미, 클래식 디저트, 고급스러움, 만족감, 화사함, 자연주의]'], 'menuIngred': ["[ '사과', '설탕 (옵션)', '물', '얼음']", "[ '홍차, 청포도 시럽, 설탕, 레몬즙, 얼음, 청포도']", "[ '블루베리, 크림치즈, 발효버터, 허브(민트 또는 바질), 치아바타']", "[ '라임'

127.0.0.1 - - [06/Jan/2025 10:39:21] "POST /sanggueon-only HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:39:21] "POST /sanggueon-gongtong HTTP/1.1" 200 -


[15, 14, 14, 14, 14]['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차']
{'status': 'success', 'onlyMenus': ['자스민 티', '디카페인 카라멜 마끼야또', '에스프레소 라떼', '바닐라 라떼 콜드브루', '말차'], 'sang1': '광안동', 'sang2': '동명동', 'sang1MenuCnt': [2, 4, 8, 6, 7], 'sang2MenuCnt': [102, 46, 46, 39, 39], 'menuInfo': ['[우아함, 허브향, 편안함, 자연스러움, 깔끔함, 은은한 꽃향, 부드러운 허브감, 우아한 풍미, 고요한 여운, 자연스러운 감미]', '[달콤함, 크리미함, 건강함, 부드러움, 만족감, 부드러운 캐러멜, 크리미한 텍스처, 가벼운 풍미, 달콤한 여운, 편안함]', '[진한 커피맛, 부드러움, 고급스러움, 풍미, 클래식, 진한 커피 맛, 부드러운 우유, 균형감, 풍부한 여운]', '[부드러움, 달콤함, 크리미함, 시원함, 고급스러움, 부드러운 바닐라 향, 깔끔한 커피, 청량감, 자연스러운 단맛, 균형 잡힌 맛]', '[쌉쌀함, 고소함, 전통적, 자연스러움, 깊은 맛, 전통적 풍미, 자연스러운 깊이, 깔끔한 여운]'], 'menuIngred': ["[ '자스민 티 잎', '뜨거운 물', '설탕', '레몬']", "[ '디카페인 커피 원두', '우유', '카라멜 시럽', '얼음']", "[ '에스프레소', '우유', '설탕 (옵션)', '바닐라 시럽 (옵션)']", "[ '콜드브루 커피', '우유', '바닐라 시럽', '얼음', '휘핑크림 (옵션)']", "[ '말차 가루', '우유', '설탕', '물', '바닐라 시럽 (옵션)']"]}

[57, 25, 37, 55, 27]
['패션후르츠 에이드', '딸기 요거', '망고 스무디', '얼그레이 밀크티', '유자 에이드']
{'status': 'success', 'gongtongMenus': ['패션후르

127.0.0.1 - - [06/Jan/2025 10:40:42] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:40:42] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:41:31] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:41:31] "POST /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:47:24] "OPTIONS /send-coordinates HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2025 10:47:24] "POST /send-coordinates HTTP/1.1" 200 -
